In [2]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [4]:
#function that takes in a 3x3 array of coordinates for faces and returns triangles and vertices
def index_unique_rows(full_coordinate_array):
    """
    Separates an array of nested coordinate rows into an array of unique rows and and index array.
    """
    vertices, flat_idx = np.unique(full_coordinate_array.reshape(-1, full_coordinate_array.shape[-1]), axis=0, return_inverse=True)
    return vertices, flat_idx.reshape(-1, full_coordinate_array.shape[-2])


In [6]:
ta3p100.LabelKey()

numeric numeric label of the compartment,description descriptive name of the label,color representative color of the compartment
0,not_labeled,no_color
1,label_removed,no_color
2,Apical,blue
3,Basal,yellow
4,Oblique,green
5,Soma,red
6,Axon-Soma,aqua
7,Axon-Dendr,off blue
8,Dendrite,purple
9,Distal,pink


In [34]:
#will take in and populate the soma table based on the key it gets
def axon_verts_faces(query_key):
    
    table=""
    vertices_soma,triangles_soma = (ta3p100.CompartmentFinal.ComponentFinal() & query_key
                                   & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]).fetch("vertex_indices","triangle_indices")

    if len(vertices_soma) > 0:
        print("Soma found in Exhitatory")
        #get the regular mesh from CleansedMesh
        vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")
    else:
        vertices_soma,triangles_soma = (ta3p100.CompartmentOrphan.ComponentOrphan() & query_key 
                                        & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]
                                       ).fetch("vertex_indices","triangle_indices")
        if len(vertices_soma) > 0:
            print("Soma found in Orphans")
            vertices_mesh,triangles_mesh = (ta3p100.CleansedMeshOrphan & query_key).fetch("vertices","triangles")
        else:
            print("No Soma exists for " + str(query_key["segment_id"]))
            return np.array([]),np.array([])
            
    ts_flatten = np.hstack(triangles_soma).astype("int64")

    vertices_real = vertices_mesh[0]
    triangles_real = triangles_mesh[0]

    ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

    vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
    return vertices_whole, triangles_whole

In [ ]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


""" New read function: for adjusted 2 vert skeleton output"""
def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)



In [41]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


""" New read function: for adjusted 2 vert skeleton output"""
def read_raw_skeleton(file_path):
    edges = list()
    with open(file_path) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        for i,row in enumerate(reader):
            v1 = (float(row[1]),float(row[2]),float(row[3]))
            v2 = (float(row[4]),float(row[5]),float(row[6]))
            edges.append((v1,v2))
    return np.array(edges).astype(float)



In [42]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/pymesh_NEURITES")) == False:
    os.mkdir("pymesh_NEURITES")

In [43]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(neuron_ID,
                                vertices=[], 
                                triangles=[],
                                folder="pymesh_NEURITES"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    
    #get the current file location
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(neuron_ID)
    path_and_filename = file_loc / filename
    
    #print(file_loc)
    #print(path_and_filename)
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename),str(filename),str(file_loc)

In [44]:
def meshlab_fix_manifold(key,folder="pymesh_NEURITES"):
    
    file_loc = pathlib.Path.cwd() / folder
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [45]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [50]:
#key = dict(segmentation=2,segment_id=648518346341353986)

key_source = dj.U("segment_id","segmentation") & (ta3p100.CompartmentFinal.ComponentFinal() 
                                     & [dict(compartment_type=x) for x in ["Axon-Dendr","Axon-Soma"]]) 

key = key_source.fetch(as_dict=True)[1]
print("key{'segment_id'} = " + str(key['segment_id']))

print()
print()
print(str(key["segment_id"])+ ":")
global_start_time = time.time()
#create return key
return_key = key.copy()


#get the vertices and triangles for the Soma
start_time = time.time()
vertices_whole, triangles_whole = axon_verts_faces(key)
print(f"Step 1: extracted Soma Mesh = {time.time()-start_time}")


#if no soma portion was found then just write regular skeleton
if not vertices_whole.any():
    print("Error no axon")
    
    
new_key = dict(segmentation=key["segmentation"],
                   segment_id=key["segment_id"])
        
        

#find the bounding box
# start_time = time.time()
# mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
# mesh.export(str(key["segment_id"]) + "_axon.off","off")

#now skeletonize the mesh

#gets all of the different parts
mesh = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
count, labels = trimesh_io.trimesh.graph.csgraph.connected_components(
                                                    mesh.edges_sparse,
                                                    directed=False,
                                                    return_labels=True)

new_key["n_bodies"] = count
values = np.array(labels)
searchval = 0
ii = np.where(values == searchval)[0]
new_key["lagest_mesh_perc"] = len(ii)/len(labels)

print(f"Step 2a: Getting the number of splits: {time.time() - start_time}")



start_time = time.time()
#pass the vertices and faces to pymeshfix to become watertight
meshfix = pymeshfix.MeshFix(vertices_whole,triangles_whole)
meshfix.repair(verbose=False,joincomp=True,remove_smallest_components=False)
print(f"Step 2b: Pymesh shrinkwrapping: {time.time() - start_time}")

#print("Step 2: Writing Off File")
start_time = time.time()
#write the new mesh to off file
path_and_filename,filename,file_loc = write_Whole_Neuron_Off_file(str(new_key["segment_id"]),meshfix.v,meshfix.f)
print(f"Step 3: Writing shrinkwrap off file: {time.time() - start_time}")

#Run the meshlabserver scripts
start_time = time.time()
output_mesh = meshlab_fix_manifold(key)
print(f"Step 4: Meshlab fixing non-manifolds: {time.time() - start_time}")

print(output_mesh[:-4])

#send to be skeletonized
start_time = time.time()
return_value = cm.calcification(output_mesh[:-4])
if return_value > 0:
    raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                    ' did not finish... exited with error code: ' + str(return_value))
#print(f"Step 5: Generating Skeleton: {time.time() - start_time}")



#read in the skeleton files into an array
bone_array = read_raw_skeleton(output_mesh[:-4]+"_skeleton.cgal")

#print(bone_array)
if len(bone_array) <= 0:
    raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
print(f"Step 5: Generating and reading Skeleton: {time.time() - start_time}")


start_time = time.time()
new_key["n_edges"] = len(bone_array)
new_key["edges"] = bone_array

np.savez(str(key["segment_id"]) + "_axon_bones.npz",bone_array=bone_array)

#new_key["branches"] = []


# new_key["time_updated"]=str(datetime.datetime.now())
# #print(key)
# #if all goes well then write to database
# self.insert1(new_key,skip_duplicates=True)
# os.system("rm "+str(path_and_filename)+"*")
# print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
# print(f"Total time: {time.time() - global_time}")
# print("\n\n")








# min_bb = np.array(mesh.bounding_box.vertices).min(0)
# max_bb = np.array(mesh.bounding_box.vertices).max(0)
# print(f"Step 2: Calculated Bounding Box = {time.time()-start_time}")

# start_time = time.time()
# #get the filtered edges according to bounding box:
# filtered_edges_postsyn = filter_edges_by_bounding_box(skeleton_data["edges"],max_bb,min_bb)

# print(f"Step 3: filtering edges = {time.time()-start_time}")



# #write off the new data to the table
# #return_key["soma_exist"] = True

# start_time = time.time()

# return_key["vertices"] = vertices_whole
# return_key["triangles"] = triangles_whole
# return_key["edges"] = filtered_edges_postsyn
# return_key["n_edges"] = filtered_edges_postsyn.shape[0]
# return_key["soma_bounding_corners"] = np.array((min_bb,max_bb))

# #         print(return_key)
# #         print(return_key.keys())
# #         print("len(return_key.keys()) = " + str(len(return_key.keys())))

# #         for k in return_key.keys():
# #             print("type(return_key["+k+"])=" + str(type(return_key[k])))

# self.insert1(return_key,skip_duplicates=True,ignore_extra_fields=True)
# print(f"Step 4: Inserted Key = {time.time()-start_time}")
# print(f"Total time = {time.time()-global_start_time}")


key{'segment_id'} = 648518346341393609


648518346341393609:
Soma found in Exhitatory
Step 1: extracted Soma Mesh = 0.22148680686950684
Step 2a: Getting the number of splits: 0.22925400733947754
Step 2b: Pymesh shrinkwrapping: 0.11857032775878906
Done writing OFF file
Step 3: Writing shrinkwrap off file: 0.06249856948852539
starting meshlabserver fixing non-manifolds
Step 4: Meshlab fixing non-manifolds: 0.727015495300293
/notebooks/14_Contacts/pymesh_NEURITES/neuron_648518346341393609_mls
Step 5: Generating and reading Skeleton: 0.4337921142578125
